# Introduction

In [1]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import requests
import folium

As said in the Data section, the source of geospatial data has been transformed from shapefile polygons into a json file that contains the coordinates of the centroids of said polygons.

The following lines will get the json file from my github repository and turn them into a pandas dataframe

In [2]:
url='https://raw.githubusercontent.com/Fidaeic/IBM-capstone/master/Geospatial%20data/Centroides_Barrios_Malaga.json'
archivo = requests.get(url).json()

In [5]:
malaga_venues=archivo['features']
neighbourhood=pd.DataFrame()
for data in malaga_venues:
    id_neighbourhood  = data['properties']['NUMBARRIO'] 
    name_neighbourhood = data['properties']['NOMBARRIO']
        
    latlon = data['geometry']['coordinates']
    lat = latlon[1]
    lon = latlon[0]
    
    neighbourhood = neighbourhood.append({'Num_neighbourhood': id_neighbourhood,
                                          'Name': name_neighbourhood.lower().capitalize(),
                                          'Latitude': lat,
                                          'Longitude': lon}, ignore_index=True)
neighbourhood["Num_neighbourhood"]=neighbourhood["Num_neighbourhood"].map(lambda x: int(x))
neighbourhood.head()

,Latitude,Longitude,Name,Num_neighbourhood
0,36.651810,-4.514196,Cañada de ceuta,307
1,36.662529,-4.513345,San juan-el albaricocal,321
2,36.735346,-4.427377,La rosaleda,109
3,36.739066,-4.428161,503 viviendas,378
4,36.740912,-4.429205,La palmilla,107


As seen in week 3 of this module, we can use a function to retrieve information on venues from Foursquare. Particularly, this function will filter by category, adding the following string to the url: '4d4b7105d754a06374d81259'.

This category corresponds to restaurants and cafés.

In [11]:
def getNearbyFoodPlaces(names, latitudes, longitudes, radius=500, LIMIT=100):
    category='4d4b7105d754a06374d81259'
    CLIENT_ID = 'MQ2VBTWEJB22CCJS4SLRXEOZPXQ5GOUMQXOCSNW5Z02V1RV4' # your Foursquare ID
    CLIENT_SECRET = 'MJ43MP3Y1J5ZQLSNQP2UVWWMF3N05YIYUR1ORYBOJZTKUJ2J' # your Foursquare Secret
    VERSION = '20180605' # Foursquare API version
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT,
                category)
        try:
                
            results = requests.get(url).json()["response"]['groups'][0]['items']

            venues_list.append([(
                name, 
                lat, 
                lng,
                v['venue']['id'],
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        except:
            pass

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighb', 
                      'Latitude', 
                      'Longitude',
                      'IdVenue',
                      'Venue', 
                      'Lat_venue', 
                      'Long_venue', 
                      'Category']
        
    
    return(nearby_venues)

In [ ]:
mlg_food_venues = getNearbyFoodPlaces(neighbourhood['Name'],
                   neighbourhood['Latitude'],
                   neighbourhood['Longitude'])

Cañada de ceuta
San juan-el albaricocal
La rosaleda
503 viviendas
La palmilla
Virreina
Herrera oria
Virreina alta
26 de febrero
Olletas
Jardin de la abadia
Haza del campillo
La consula
Seminario
Ensanche centro
Carranque
Cruz del humilladero
Parque las virreinas
El coronel
Campo de golf
Pizarrillo
Sierra blanquilla
Jardin de malaga
El palo
La cizaña
Parq emp perez texeira
Recinto ferial cortijo torres
Parq emp la estrella
Sagrada familia
Las erizas
Las cuevas
Miramar del palo
La pelusa
Centro historico
La merced
San felipe neri
Capuchinos
Camino de olias
El ejido
Lagunillas
Vega de oro
Campamento benitez
Miraflores
Playa virginia
Pol alameda
Cortijo de maza
Los chopos
Las niñas
Carambuco
Hacienda sanchez blanca
La estacion
La malagueta
Perchel sur
La aurora
Los tilos
Nucleo gral franco
4 de diciembre
Ntra sra del carmen
Los prados
La victoria
Nuevo san andres 1
Campos eliseos
Monte de gibralfaro
Pol ctra de cartama
La barriguilla
La pelusilla
Explanada de la estación
Finca el candado
L

The outcome of the previous function has been stored in my github repository under the name 'mlg_food_venues.csv', so the API key is not displayed publicly.

In [10]:
mlg_food_venues=pd.read_csv('https://raw.githubusercontent.com/Fidaeic/IBM-capstone/master/Data/mlg_food_venues.csv')
column_names = mlg_food_venues.

,Barrio
0,Cañada de ceuta
1,Cañada de ceuta
2,San juan-el albaricocal
3,San juan-el albaricocal
4,San juan-el albaricocal
